In [4]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn
from datetime import datetime, timedelta

%matplotlib inline

# Parameters
VOL_PERIOD = 13
RETURN_PERIOD_1 = 63
RETURN_PERIOD_2 = 126
WEIGHTS = (.5,.3,.2)

portfolio = {
    'equity':("VTI", "VEU", "SHY"), 
    'credit':("HYG", "CIU", "SHY"), 
    'real estate':("REM", "VNQ", "SHY"), 
    'stress':("GLD", "TLT", "SHY")
}

In [5]:
def overall_weight(row):
    return WEIGHTS[0]*row['Period 1 Return Rank'] + WEIGHTS[1]*row['Period 2 Return Rank'] + WEIGHTS[2]*row['Std Dev Rank']

def make_table(sym):
    end = datetime.today()
    lookback = 2 * max(RETURN_PERIOD_1, RETURN_PERIOD_2)
    start = end - timedelta(lookback)
    df = web.DataReader(sym, 'yahoo', start, end)
    df.drop(["Open", "High", "Low", "Close", "Volume"], axis=1, inplace=True)
    df['Std Dev'] = pd.rolling_std(df['Adj Close'], VOL_PERIOD)
    df['Period 1 Return'] = df.pct_change(RETURN_PERIOD_1)['Adj Close']
    df['Period 2 Return'] = df.pct_change(RETURN_PERIOD_2)['Adj Close']
    return df

def get_latest(sym):
    df = make_table(sym).tail(1)
    df.index = [sym]
    return df

def cluster_table(symbols):
    dfs = map(get_latest, symbols)
    cdf = pd.concat(dfs, axis=0)
    cdf_r = cdf.rank('rows', ascending=False)[['Period 1 Return', 'Period 2 Return']]
    cdf_r['Std Dev'] = cdf.rank('rows')[['Std Dev']]
    df = pd.merge(cdf, cdf_r, left_index=True, right_index=True, how='inner', suffixes=('', ' Rank'))
    overall_df = pd.DataFrame(df.apply(overall_weight, axis=1), columns=["Overall Weight Rank"]).rank('rows')
    df = pd.merge(df, overall_df, left_index=True, right_index=True, how='inner')
    df = df.sort_index(axis=1)
    df = df.sort_values('Overall Weight Rank')
    return df
    

In [6]:
# Equity Model
cluster_table(portfolio['equity'])

,Adj Close,Overall Weight Rank,Period 1 Return,Period 1 Return Rank,Period 2 Return,Period 2 Return Rank,Std Dev,Std Dev Rank
SHY,84.940002,1,0.002572,1,0.002927,1,0.062228,1
VTI,103.839996,2,-0.045291,2,-0.037055,2,2.492487,3
VEU,45.480000,3,-0.068741,3,-0.089516,3,1.465865,2


In [7]:
# Credit Model
cluster_table(portfolio['credit'])

,Adj Close,Overall Weight Rank,Period 1 Return,Period 1 Return Rank,Period 2 Return,Period 2 Return Rank,Std Dev,Std Dev Rank
CIU,108.809998,1.5,0.009179,1,-0.005839,2,0.239379,2
SHY,84.940002,1.5,0.002572,2,0.002927,1,0.062228,1
HYG,84.949997,3.0,-0.029999,3,-0.042341,3,1.094338,3


In [8]:
# Real Estate Model
cluster_table(portfolio['real estate'])

,Adj Close,Overall Weight Rank,Period 1 Return,Period 1 Return Rank,Period 2 Return,Period 2 Return Rank,Std Dev,Std Dev Rank
SHY,84.940002,1,0.002572,2,0.002927,1,0.062228,1
VNQ,79.550003,2,0.029669,1,-0.017344,2,1.843746,3
REM,10.410000,3,-0.019356,3,-0.075414,3,0.216362,2


In [9]:
# Econ Stress Model
cluster_table(portfolio['stress'])

,Adj Close,Overall Weight Rank,Period 1 Return,Period 1 Return Rank,Period 2 Return,Period 2 Return Rank,Std Dev,Std Dev Rank
TLT,123.209999,1,0.068356,1,-0.045284,3,0.865552,2
SHY,84.940002,2,0.002572,3,0.002927,1,0.062228,1
GLD,111.309998,3,0.005147,2,-0.027351,2,1.409318,3
